In [1]:
import numpy as np

In [2]:
class Filter:
    def __init__(self, width, height, depth):
        self.weights = np.random.uniform(-1e-4, 1e-4, (depth, height, width))
        self.bias = 0
        self.weights_grad = np.zeros(self.weights.shape)
        self.bias_grad = 0
    def __repr__(self):
        return "filter weights=\n{}\nbias=\n{}".format(repr(self.weights), repr(self.bias))
    def get_weights(self):
        return self.weights
    def get_bias(self):
        return self.bias
    def update(self, learning_rate):
        self.weights -= learning_rate * self.weights_grad
        self.bias -= learning_rate * self.bias_grad

In [3]:
def padding(input_array, zp):
    if zp == 0:
        return input_array
    else:
        if input_array.ndim == 3:
            input_width = input_array.shape[2]
            input_height = input_array.shape[1]
            input_depth = input_array.shape[0]
            padded_array = np.zeros((input_depth, input_height+2*zp, input_width+2*zp))
            padded_array[:, zp : zp+input_height, zp : zp+input_width] = input_array
            return padded_array
        elif input_array.ndim == 2:
            input_width = input_array.shape[1]
            input_height = input_array.shape[0]
            padded_array = np.zeros((input_height+2*zp, input_width+2*zp))
            padded_array[zp : zp+input_height, zp : zp+input_width] = input_array
            return padded_array

In [8]:
a=np.array([[1,2,3],[4,5,6]])
np.unravel_index(a.argmax(), a.shape)

(1, 2)

In [4]:
def get_patch(input_array, i, j, filter_width, filter_height, stride):
    start_i = i * stride
    start_j = j * stride
    if input_array.ndim == 2:
        return input_array[start_i:start_i+filter_height, start_j:start_j+filter_width]
    elif input_array.ndim == 3:
        return input_array[:, start_i:start_i+filter_height, start_j:start_j+filter_width]

In [5]:
def get_max_index(array):
    return np.unravel_index(array.argmax(), array.shape)
    

In [6]:
def conv(input_array, kernel_array, output_array, stride, bias):
    channel_number = input_array.ndim
    output_width = output_array.shape[1]
    output_height = output_array.shape[0]
    kernel_width = kernel_array.shape[-1]
    kernel_height = kernel_array.shape[-2]
    for i in range(output_height):
        for j in range(output_width):
            output_array[i][j] = (get_patch(input_array, i, j, kernel_width, \
                                  kernel_height, stride) * kernel_array).sum() + bias

In [7]:
def element_wise_op(array, op):
    for i in np.nditer(array, op_flags=["readwrite"]):
        i[...] = op(i)

In [8]:
class Relu:
    def forward(self, weight_input):
        return max(0, weight_input)
    def backward(self, ouput):
        return 1 if ouput > 0 else 0

In [9]:
a=np.array([[1,-2,3],[4,6,-1]])
ar= Relu()
element_wise_op(a,ar.backward)
a

array([[1, 0, 1],
       [1, 1, 0]])

In [ ]:
class ConvLayer:
    def __init__(self, input_width, input_height, channel_number, filter_width, \ 
                 filter_height, filter_number, zero_padding, stride, activator, learnig_rate):
        self.input_width = input_width
        self.input_height = input_height
        self.channel_number = channel_number
        self.filter_width = filter_width
        self.filter_height = filter_height
        self.filter_number = filter_number
        self.zero_padding = zero_padding
        self.stride = stride
        self.output_width = ConvLayer.calc_output_size(input_width, filter_width)
        self.output_height = ConvLayer.calc_output_size(input_height, filter_height)
        self.output_array = np.zero((filter_number, self.output_height, self.output_width))
        self.filters = []
        for _ in range(filter_number):
            self.filters.append(Filter(filter_width, filter_height, self.channel_number))
        self.activator = activator
        self.learnig_rate = learnig_rate
        @classmethod
        def calc_output_size(cls, input_szie, filter_size):
            return (input_szie-filter_size+2*cls.zero_padding) / cls.stride +1
        def forward(self, input_array):
            self.input_array = input_array
            self.padded_input_array = padding(input_array, self.zero_padding)
            for i in range(self.filter_number):
                filter = self.filters[i]
                conv(self.padded_input_array, filter.get_weight(), self.output_array[i], \
                     self.stride, filter.get_bias())
                element_wise_op(self.output_array, self.activator.forward)
        def expand_sensitivity_map(self, sensitivity_array):
            depth = sensitivity_array.shape[0]
            expanded_width = (self.input_width-self.filter_width+2*self.zero_padding+1)
            expanded_height = (self.input_height-self.filter_height+2*self.zero_padding+1)
            expand_array = np.zeros((depth, expanded_height, expanded_width))
            for i in range(self.output_height):
                for j in range(self.output_width):
                    i_pos = i * self.stride
                    j_pos = j * self.stride
                    expand_array[:,i_pos,j_pos] = sensitivity_array[:,i,j]
            return expand_array
        def bp_sensitivity_map(self, sensitivity_array, activator):

        def backward(self, input_array, sensitivity_array, activator):
            self.forward(input_array)
            self.